In [ ]:
# This notebook prepares froam a csv file (sample kaggle data: https://www.kaggle.com/datasets/ishanshrivastava28/tata-online-retail-dataset) and inserts it into a SQL table 
# It serves as a starting point for a simulated, quasi-real business case focused on building a simple database (Data Warehouse, Star Schema).

In [ ]:
import pyodbc
import pandas as pd
import numpy as np
import names



In [ ]:
# Step 1: Read the CSV file into a pandas DataFrame
df = pd.read_csv('data/Online_Retail_Data_Set.csv')

# Step 2: Add random Client and Email columns

unique_ids = df['CustomerID'].unique()
n_unique = len(unique_ids)

# Pre-generate a pool of names
pool_size = n_unique
first_names_pool = [names.get_first_name() for _ in range(n_unique)]
last_names_pool = [names.get_last_name() for _ in range(n_unique)]

# Use np.random.choice to vectorize the selection for each row
first_names_unique = np.random.choice(first_names_pool, size=n_unique)
last_names_unique = np.random.choice(last_names_pool, size=n_unique)

clients_unique = pd.Series(first_names_unique) + ' ' + pd.Series(last_names_unique)
emails_unique = pd.Series(first_names_unique).str.lower() + '.' + pd.Series(last_names_unique).str.lower() + '@example.com'

# Build mapping dictionaries from CustomerID to client name and email
client_map = dict(zip(unique_ids, clients_unique))
email_map = dict(zip(unique_ids, emails_unique))


# Map these values back to the original DataFrame so that each CustomerID gets consistent values
df['Client'] = df['CustomerID'].map(client_map)
df['Email'] = df['CustomerID'].map(email_map)

# Step 3: Clean the DataFrame
# Convert InvoiceDate column to datetime; invalid parsing will result in NaT
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# Ensure numeric columns are properly converted
df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce', downcast='integer')

# Convert CustomerID to string (NVARCHAR expected); this will turn missing values into 'nan'
df['CustomerID'] = df['CustomerID'].apply(lambda x: str(int(float(x))) if pd.notnull(x) else None)
# Replace pandas NaN with Python None so that pyodbc will insert SQL NULLs
df = df.where(pd.notnull(df), None)

# Select the columns in the order of the table definition
columns = ['InvoiceNo', 'StockCode', 'DescriptionProd', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'Client', 'Email']

# Convert the DataFrame rows to a list of tuples
data_to_insert = list(df[columns].itertuples(index=False, name=None))

In [ ]:
# Step 4: Create table schema in SQL Server


# Define connection string to SQL Server
conn_str = (
    "DRIVER={SQL Server};"
    "SERVER=maciek_d;"
    "DATABASE=retail_online;"
    "Trusted_Connection=yes;"
)

# Establish the connection
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Drop the table if it already exists (this removes the old schema and data)
drop_table_sql = "IF OBJECT_ID('dbo.OrginalData', 'U') IS NOT NULL DROP TABLE dbo.OrginalData;"
cursor.execute(drop_table_sql)
conn.commit()

# Create the new table with the desired schema
create_table_sql = """
CREATE TABLE OrginalData (
    InvoiceNo NVARCHAR(50),
    StockCode NVARCHAR(50),
    DescriptionProd NVARCHAR(255),
    Quantity INT,
    InvoiceDate DATETIME,
    UnitPrice DECIMAL(10,2),
    CustomerID NVARCHAR(50),
    Country NVARCHAR(100),
    Client NVARCHAR(50),
    Email NVARCHAR(50)


);
"""
cursor.execute(create_table_sql)
conn.commit()

In [ ]:
# --- Step 5: Insert Data Directly from DataFrame to SQL Server ---

# Prepare the INSERT statement with placeholders
insert_sql = """
INSERT INTO OrginalData (InvoiceNo, StockCode, DescriptionProd, Quantity, InvoiceDate, UnitPrice, CustomerID, Country, Client, Email)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
"""

# Enable fast executemany for better performance on large datasets
cursor.fast_executemany = True

# Execute the bulk insert
cursor.executemany(insert_sql, data_to_insert)
conn.commit()

# --- Cleanup: Close the Connection ---
cursor.close()
conn.close()